In [1]:
import torch
import torch.nn as nn

import tensorflow as tf
import numpy as np

from tqdm import tqdm
from nobuco import pytorch_to_keras, ChannelOrder

from model.TwinLite import TwinLiteNet
from DataSet import CustomDataset

2024-08-20 10:43:34.156098: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-20 10:43:34.156126: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-20 10:43:34.156915: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-20 10:43:34.162697: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-20 10:43:35.114843: W tensorflow/compiler/tf2

In [2]:
state_dict = torch.load('pretrained/epo99_qat.pth', map_location='cpu')

model = nn.Sequential(torch.ao.quantization.QuantStub(), TwinLiteNet(), torch.ao.quantization.DeQuantStub())
model.load_state_dict(state_dict)
model = model[1]
model.eval()

/home/ubuntadmin/Desktop/TwinLiteNet/venv/lib/python3.11/site-packages/nobuco/trace/trace.py:366: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  outputs = orig_method(*args, 

TwinLiteNet(
  (encoder): ESPNet_Encoder(
    (level1): CBR(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act): PReLU(num_parameters=16)
    )
    (sample1): InputProjectionA(
      (pool): ModuleList(
        (0): AvgPool2d(kernel_size=3, stride=2, padding=1)
      )
    )
    (sample2): InputProjectionA(
      (pool): ModuleList(
        (0-1): 2 x AvgPool2d(kernel_size=3, stride=2, padding=1)
      )
    )
    (b1): CBR(
      (conv): Conv2d(19, 19, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(19, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act): PReLU(num_parameters=19)
    )
    (level2_0): DownSamplerB(
      (c1): C(
        (conv): Conv2d(19, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      )
      (d1): CDilated(
        (conv): Conv2d(12, 16,

In [3]:
# TODO, CHANGE FOR YOUR OWN DATASET DIR
DATASET_PATH = "/media/ubuntadmin/67b3a250-6e0f-4a86-b15d-1b302250e0b4/home/ubuntadmin/Desktop/data/datasets/road-segmentation/TwinLiteTree/images"
MAX_CALIBRATION_IMG = 10
dataset = CustomDataset(DATASET_PATH, valid=True)
dataset.names = dataset.names[:MAX_CALIBRATION_IMG]

In [4]:
input = torch.randn(1, 3, dataset.H_, dataset.W_)
keras_model = pytorch_to_keras(model, [input], inputs_channel_order=ChannelOrder.TENSORFLOW)

[Nobuco] Tracing (DONE): 216 ops [00:04]
[Nobuco] Converting: |          | 2/162 ops [00:03]

2024-08-20 10:43:56.189338: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


[Nobuco] Converting: |██████▌   | 107/162 ops [00:07]

2024-08-20 10:44:00.928166: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51840000 exceeds 10% of free system memory.


[Nobuco] Converting: |███████   | 114/162 ops [00:08]

2024-08-20 10:44:01.892658: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51840000 exceeds 10% of free system memory.
2024-08-20 10:44:01.922141: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51840000 exceeds 10% of free system memory.
2024-08-20 10:44:01.951064: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51840000 exceeds 10% of free system memory.
2024-08-20 10:44:01.967234: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51840000 exceeds 10% of free system memory.


[Nobuco] Converting (DONE): |██████████| 162/162 ops [00:13]
Legend:
    Green — conversion successful
    Yellow — conversion imprecise
    Red — conversion failed
    Red — no converter found
    Bold — conversion applied directly
    * — subgraph reused
    Tensor — this output is not dependent on any of subgraph's input tensors
    Tensor — this input is a parameter / constant
    Tensor — this tensor is useless

TwinLiteNet[model.TwinLite](float32_0<1,3,360,640>) -> (float32_299<1,2,360,640>, float32_326<1,2,360,640>)
 │  ESPNet_Encoder[model.TwinLite](float32_0<1,3,360,640>) -> float32_272<1,32,45,80>
 │   │  CBR[model.TwinLite](float32_0<1,3,360,640>) -> float32_9<1,16,180,320>
 │   │   │  Conv2d[torch.nn.modules.conv](float32_0<1,3,360,640>) -> float32_2<1,16,180,320>
 │   │   │   └· conv2d[torch.nn.functional](float32_0<1,3,360,640>, float32_1<16,3,3,3>, None, (2, 2), (1, 1), (1, 1), 1) -> float32_2<1,16,180,320>
 │   │   │  BatchNorm2d[torch.nn.modules.batchnorm](float32_2<1,

In [5]:
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)

tflite_model = converter.convert()
with open(f"models/twinlitenet.tflite", 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpbio28vqj/assets


INFO:tensorflow:Assets written to: /tmp/tmpbio28vqj/assets
2024-08-20 10:44:29.645848: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-08-20 10:44:29.645879: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-08-20 10:44:29.646675: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpbio28vqj
2024-08-20 10:44:29.671011: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-08-20 10:44:29.671040: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpbio28vqj
2024-08-20 10:44:29.719476: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-08-20 10:44:29.741840: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-08-20 10:44:30.101655: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [6]:
def representative_dataset_gen():
	for img_name, input, (seg_da, seg_ll) in tqdm(dataset):
		input = input.float() / 255.0
		input = input.numpy()
		input = np.expand_dims(input, axis=0)
		input = np.transpose(input, (0, 2, 3, 1))
		yield [input]
		
qt_converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
qt_converter.optimizations = [tf.lite.Optimize.DEFAULT]
qt_converter.representative_dataset = representative_dataset_gen

tflite_model = qt_converter.convert()
with open(f"models/twinlitenet_qt.tflite", 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp_erm9o0m/assets


INFO:tensorflow:Assets written to: /tmp/tmp_erm9o0m/assets
/home/ubuntadmin/Desktop/TwinLiteNet/venv/lib/python3.11/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-08-20 10:44:50.171584: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-08-20 10:44:50.171636: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-08-20 10:44:50.172007: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp_erm9o0m
2024-08-20 10:44:50.206305: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-08-20 10:44:50.206336: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp_erm9o0m
2024-08-20 10:44:50.284272: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
